In [1]:
import h2o
from h2o.estimators.word2vec import H2OWord2vecEstimator
import json
import pandas as pd
import numpy as np
import networkx as nx
import jellyfish
import os
import shutil
import subprocess
import requests
from github import Github
from git import Repo
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from zipfile import ZipFile
from filecmp import dircmp
import configparser
from pprint import pprint
from datetime import datetime
import random
from feature_engine.imputation import MeanMedianImputer
from sklearn.cluster import KMeans

In [2]:
config = configparser.ConfigParser()
config.read('authentication.ini')
print(config.sections())
github_oauth = config['github']['api_key']

['github']

In [3]:
project_name = 'Redisson'
project_owner = 'redisson'
project_name_short = 'redisson'

In [4]:
# Parameters
project_name = "karaf"
project_owner = "apache"
project_name_short = "karaf"


In [5]:
okhttp_commits_df = pd.read_csv(f'Effort_Estimation_Results_v6/{project_name}.csv')

In [6]:
okhttp_commits_df.head()

,kmean_label,time_taken_mean,time_taken_min,time_taken_max,time_taken_q10,time_taken_q20,time_taken_q25,time_taken_q30,time_taken_q40,time_taken_q50,...,num_dependency_max,num_dependency_std,num_line_affected_mean,num_line_affected_min,num_line_affected_max,num_line_affected_std,actual_num_of_classes_touched_mean,actual_num_of_classes_touched_min,actual_num_of_classes_touched_max,actual_num_of_classes_touched_std
0,0,24.250000,1.0,164.0,1.0,2.0,3.0,4.0,7.0,13.0,...,64.0,6.363961,4.0,4,4,0.0,4.0,4,4,0.0
1,0,24.250000,1.0,164.0,1.0,2.0,3.0,4.0,7.0,13.0,...,17.0,7.571878,35.0,35,35,0.0,2.0,2,2,0.0
2,13,25.155606,1.0,163.0,1.0,2.0,3.0,4.0,8.0,13.0,...,12.0,0.707107,48.0,48,48,0.0,2.0,2,2,0.0
3,14,23.773360,1.0,160.0,1.0,2.0,3.0,4.0,6.0,11.0,...,12.0,0.707107,50.0,50,50,0.0,2.0,2,2,0.0
4,14,23.773360,1.0,160.0,1.0,2.0,3.0,4.0,6.0,11.0,...,18.0,NaN,169.0,169,169,NaN,2.0,2,2,NaN


In [7]:
len(okhttp_commits_df)

5

In [8]:
okhttp_commits_df['commit_compared_with']

0    c373315a5b53fa15ccb5322f7d40eda235d2a4c8
1    0f20737512ac120452e2749b7e41f6857488e9d2
2    e7201d42fc4e9a13b32c9f42dda5ae40a9648be5
3    91f2082b66ad2d6ce5f3731912c5117ef10f248c
4    8c2d5f928b3ddd3b9786e9a6b0b37a38dddaf974
Name: commit_compared_with, dtype: object

In [9]:
okhttp_commits_df['num_line_affected_max']

0      4
1     35
2     48
3     50
4    169
Name: num_line_affected_max, dtype: int64

In [10]:
def get_commit_line_changed(sha, project_owner, project_name_short):
    query_url = f"https://api.github.com/repos/{project_owner}/{project_name_short}/commits/{sha}"

    headers = {'Authorization': f'token {github_oauth}'}
    r = requests.get(query_url, headers=headers)
    #print((r.json()))
    return r.json()['stats']['total']

In [11]:
def limit_perc_change(perc):
    if perc > 1:
        return 1
    else:
        return perc

In [12]:
def limit_base_time(time):
    if time < 1:
        return 1
    else:
        return time

In [13]:
get_commit_line_changed('083315a473d8b2fc972a70449272b296683ac38c', 'square', 'okhttp')

358

In [14]:
okhttp_commits_df['commit_line_changed'] = okhttp_commits_df.apply(lambda row: get_commit_line_changed(row['commit_compared_with'], project_owner, project_name_short), axis=1)

In [15]:
okhttp_commits_df['num_line_affected_min']

0      4
1     35
2     48
3     50
4    169
Name: num_line_affected_min, dtype: int64

In [16]:
okhttp_commits_df['commit_line_changed']

0     85
1     12
2    142
3      8
4      3
Name: commit_line_changed, dtype: int64

In [17]:
okhttp_commits_df['refactoring_perc'] = okhttp_commits_df['num_line_affected_min'] / okhttp_commits_df['commit_line_changed']

In [18]:
okhttp_commits_df['refactoring_perc'] = okhttp_commits_df.apply(lambda row: limit_perc_change(row['refactoring_perc']), axis=1)

In [19]:
okhttp_commits_df['refactoring_perc']

0    0.047059
1    1.000000
2    0.338028
3    1.000000
4    1.000000
Name: refactoring_perc, dtype: float64

In [20]:
okhttp_commits_df['refactoring_perc_time_taken'] = okhttp_commits_df['refactoring_perc'] * okhttp_commits_df['time_taken']

In [21]:
okhttp_commits_df['refactoring_perc_time_taken'] = okhttp_commits_df.apply(lambda row: limit_base_time(row['refactoring_perc_time_taken']), axis=1)


In [22]:
okhttp_commits_df['refactoring_perc_time_taken']

0      1.000000
1     15.000000
2     24.676056
3     11.000000
4    140.000000
Name: refactoring_perc_time_taken, dtype: float64

In [23]:
okhttp_commits_df['time_taken']

0     18.0
1     15.0
2     73.0
3     11.0
4    140.0
Name: time_taken, dtype: float64

In [24]:
for column in okhttp_commits_df.columns:
    print(column)

kmean_label

time_taken_mean

time_taken_min

time_taken_max

time_taken_q10

...

actual_num_of_classes_touched_max

actual_num_of_classes_touched_std

commit_line_changed

refactoring_perc

refactoring_perc_time_taken

In [25]:
okhttp_commits_df.to_csv(f'Effort_Estimation_Results_3E/{project_name}.csv', index=False)